In [14]:
import numpy as np 
from scipy.linalg import svd 
import pdal 
import os
import time

In [7]:
filename = "/media/riannek/minimax/gleis/templates/befahren.las"
if not os.path.exists(filename):
    raise FileNotFoundError

In [8]:
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()

In [ ]:
def pca(cloud):
    """Use PCA to get einvalues and eigenvectors of a point cloud"""
    mean = np.mean(cloud, axis=0)
    centered = cloud - mean
    cov_matrix = np.cov(centered, rowvar=False) # row variance nicht berechnen
    eigenvals, eigenvecs = np.linalg.eig(cov_matrix)
    sorted_indices = np.argsort(eigenvals)[::-1]
    sorted_eigenvals = eigenvals[sorted_indices]
    sorted_eigenvecs = eigenvecs[:,sorted_indices]
    # Returned vectors are in columns, first vector is eigenvec[:, 0] == eigenvec.T[0]
    return sorted_eigenvals, sorted_eigenvecs

In [ ]:
def pca_svd(cloud):
    """Use PCA to get einvalues and eigenvectors of a point cloud"""
    mean = np.mean(cloud, axis=0)
    centered = cloud - mean
    U, S, Vt = svd(centered, full_matrices=False)
    eigenvals = S**2/(cloud.shape[0]-1)
    # Returned vectors are in columns, first vector is eigenvec[:, 0] == eigenvec.T[0]
    return eigenvals, Vt.T

In [21]:
start = time.time()
eigenvals, eigenvecs = pca(xyz)
print("PCA took", time.time()-start, "seconds")
print(eigenvals)
print(eigenvecs)

PCA took 0.0019104480743408203 seconds
[5.76496067e+00 3.96052162e-04 1.77689797e-04]
[[ 0.99998133  0.00602064 -0.0010479 ]
 [ 0.00304713 -0.63986045 -0.76848508]
 [ 0.00529728 -0.76846754  0.63986685]]


In [20]:
start = time.time()
eigenvals, eigenvecs = pca_svd(xyz)
print("PCA SVD took", time.time()-start, "seconds")
print(eigenvals)
print(eigenvecs)

PCA SVD took 0.0006365776062011719 seconds
[5.76496067e+00 3.96052162e-04 1.77689797e-04]
[[ 0.99998133 -0.00602064 -0.0010479 ]
 [ 0.00304713  0.63986045 -0.76848508]
 [ 0.00529728  0.76846754  0.63986685]]


In [23]:
delta = 0.0019104480743408203 - 0.0006365776062011719

In [27]:
delta * 300 * 1000 / 60 # Könnte ein paar Minuten einsparen

6.369352340698242